<a href="https://colab.research.google.com/github/larissamazzero/RMN/blob/main/PLS/pls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Partial Least Squares Regression (PLS)

##Definição

A regressão PLS (partial least squares) vem sendo amplamente utilizada em procedimentos de seleção de variáveis por sua capacidade de operar com grande número de variáveis correlacionadas e afetadas por ruído. 

Este artigo propõe um método para identificar o melhor subconjunto de variáveis de processo para a predição das variáveis de resposta. É uma técnica que reduz os preditores a um conjunto menor de componentes não correlacionados e efetua regressão de mínimos quadrados para esses componentes no lugar dos dados originais. 

A regressão PLS é particularmente útil quando os preditores são altamente colineares ou quando existem mais preditores do que observações e a regressão de mínimos quadrados ordinários gera coeficientes com erros padrão altos ou que falha totalmente.

##Casos de uso

A regressão PLS é usada primariamente nos setores químico, farmacêutico, alimentício e plástico. Uma aplicação comum consiste em modelar a relação entre medidas espectrais (NIR, IR e UV), que incluem muitas variáveis frequentemente correlacionadas entre si, e composição química ou outras propriedades físico-químicas. Na regressão PLS, a ênfase está em desenvolver modelos preditivos. Por isso ela não é muito usada para filtrar variáveis que não sejam úteis para explicar a resposta.

FONTE: https://support.minitab.com/pt-br/minitab/20/help-and-how-to/statistical-modeling/regression/supporting-topics/partial-least-squares-regression/what-is-partial-least-squares-regression/

##Funcionamento

A regressão PLS relaciona a matriz X (composta por variáveis de processo x) à matriz Y (composta por variáveis de produto y), permitindo analisar dados com forte correlação, elevados níveis de ruído e desequilíbrio entre o número de variáveis e observações. Tal regressão gera um conjunto de parâmetros que fornecem informações sobre a estrutura e comportamento de X e Y, o que corrobora para sua ampla aplicação em procedimentos de seleção de variáveis (WOLD; SJÖSTRÖM; ERIKSSON, 2001).

Etapa 1: Dividir o banco de dados em conjuntos de treino e teste

Etapa 2: Aplicar a regressão PLS no conjunto de treino e gerar índices de importância das variáveis

Etapa 3: Predizer a variável de resposta y para o conjunto de treino e eliminar as variáveis irrelevantes e ruidosas

Etapa 4: Construir um gráfico para identificar o melhor subconjunto de variáveis e testar essas variáveis no conjunto de teste

Etapa 5: Comparar o desempenho dos índices de importância e identificar o melhor método para a seleção de variáveis

FONTE: https://www.scielo.br/j/prod/a/7Pbw8HKvxD7F87rTSLJqCFD/?format=pdf&lang=pt#:~:text=A%20regressão%20PLS%20consiste%20em,número%20reduzido%20de%20combinações%20lineares.

##Requisitos

##Exemplo de aplicação